In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, pearsonr, spearmanr

# Font settings
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # Prevent minus sign issues


orgDf = pd.read_csv('../../datasets/credit_card_fraud_dataset.csv')
orgDf.head()

제안 2: 이상 거래 탐지
- 문제 정의:
    +  거래 데이터(Amount, TransactionType, Location, IsFraud)를 바탕으로 이상 거래를 탐지하고, 거래 사기 가능성을 예측합니다.
- 목표:
    + 거래 금액의 분포 및 이상치 탐색
    + 거래 유형별 사기 발생률 비교
    + 사기 거래가 자주 발생하는 지역 및 시간대 식별
-   배경:
    + 이상 거래를 조기에 탐지함으로써 사기 방지 시스템을 강화하고, 금융 손실을 줄일 수 있습니다.

In [ ]:
orgDf.info(), orgDf.index, orgDf.describe()

In [ ]:
orgDf['Amount'].describe()

In [ ]:
len(orgDf['MerchantID'].value_counts()), orgDf['MerchantID'].value_counts()

In [ ]:
orgDf['IsFraud'].value_counts()

In [ ]:
orgDf['Location'].value_counts() 

In [231]:
fraud = orgDf[orgDf['IsFraud'] == 1]


In [ ]:
fraud['Location'].value_counts()

In [ ]:
rateOfPurchase = (orgDf['Location'].value_counts()) / 100000
rateOfPurchase

In [ ]:
rateOfFraudByLoc = (fraud['Location'].value_counts()) / 1000
rateOfFraudByLoc

In [ ]:
orgDf['TransactionDate'].info, orgDf['TransactionDate'].describe(), orgDf['TransactionDate'].value_counts(), orgDf['TransactionDate'].index

### 거래 금액의 분포 및 이상치 탐색

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 1. Transaction Amount Distribution Analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Box Plot (Normal Scale)
bp = ax1.boxplot(orgDf['Amount'], patch_artist=True)
ax1.set_title('Transaction Amount Distribution\n(Normal Scale)', fontsize=12)
ax1.set_ylabel('Amount')
ax1.grid(True, linestyle='--', alpha=0.7)
plt.setp(bp['boxes'], color='black', facecolor='lightsteelblue')
plt.setp(bp['medians'], color='red', linewidth=1.5)
plt.setp(bp['whiskers'], color='black', linestyle='-')
plt.setp(bp['fliers'], marker='o', markerfacecolor='gray', alpha=0.5)

# Box Plot (Log Scale)
bp = ax2.boxplot(orgDf['Amount'], patch_artist=True)
ax2.set_yscale('log')
ax2.set_title('Transaction Amount Distribution\n(Log Scale)', fontsize=12)
ax2.set_ylabel('Amount (log scale)')
ax2.grid(True, linestyle='--', alpha=0.7)
plt.setp(bp['boxes'], color='black', facecolor='lightsteelblue')
plt.setp(bp['medians'], color='red', linewidth=1.5)
plt.setp(bp['whiskers'], color='black', linestyle='-')
plt.setp(bp['fliers'], marker='o', markerfacecolor='gray', alpha=0.5)

# Histogram (Normal Scale)
n, bins, patches = ax3.hist(orgDf['Amount'], bins=50, 
                           facecolor='lightsteelblue', 
                           edgecolor='black', 
                           alpha=0.7)
ax3.set_title('Transaction Amount Histogram\n(Normal Scale)', fontsize=12)
ax3.set_xlabel('Amount')
ax3.set_ylabel('Frequency')
ax3.grid(True, linestyle='--', alpha=0.7)

# Histogram (Log Scale)
n, bins, patches = ax4.hist(orgDf['Amount'], bins=50, 
                           facecolor='lightsteelblue', 
                           edgecolor='black', 
                           alpha=0.7)
ax4.set_xscale('log')
ax4.set_title('Transaction Amount Histogram\n(Log Scale)', fontsize=12)
ax4.set_xlabel('Amount (log scale)')
ax4.set_ylabel('Frequency')
ax4.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# 2. Fraud vs Normal Transaction Comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Box Plot Comparison (Normal and Log Scale)
normal_amounts = orgDf[orgDf['IsFraud']==0]['Amount']
fraud_amounts = orgDf[orgDf['IsFraud']==1]['Amount']

# Normal Scale
data = [normal_amounts, fraud_amounts]
bp1 = ax1.boxplot(data, labels=['Normal', 'Fraud'], patch_artist=True)
ax1.set_title('Transaction Amount by Class\n(Normal Scale)', fontsize=12)
ax1.set_ylabel('Amount')
ax1.grid(True, linestyle='--', alpha=0.7)
plt.setp(bp1['boxes'][0], facecolor='lightsteelblue')
plt.setp(bp1['boxes'][1], facecolor='lightcoral')
plt.setp(bp1['medians'], color='red', linewidth=1.5)

# Log Scale
bp2 = ax2.boxplot(data, labels=['Normal', 'Fraud'], patch_artist=True)
ax2.set_yscale('log')
ax2.set_title('Transaction Amount by Class\n(Log Scale)', fontsize=12)
ax2.set_ylabel('Amount (log scale)')
ax2.grid(True, linestyle='--', alpha=0.7)
plt.setp(bp2['boxes'][0], facecolor='lightsteelblue')
plt.setp(bp2['boxes'][1], facecolor='lightcoral')
plt.setp(bp2['medians'], color='red', linewidth=1.5)

plt.tight_layout()
plt.show()

# Print statistics
print("\n=== Amount Statistics by Transaction Class ===")
print("\nNormal Transactions:")
print(normal_amounts.describe().round(2))
print("\nFraudulent Transactions:")
print(fraud_amounts.describe().round(2))


In [ ]:
'''Normal Scale (선형 스케일)
데이터를 있는 그대로의 값으로 표현
일정한 간격으로 눈금이 표시됨 (예: 0, 100, 200, 300, ...)
장점: 직관적이고 실제 값을 바로 파악 가능
단점: 값의 범위가 매우 클 경우 작은 값들의 차이를 구분하기 어려움
Log Scale (로그 스케일)
데이터를 로그 변환하여 표현
지수적으로 눈금이 증가 (예: 1, 10, 100, 1000, ...)
장점:
매우 큰 값과 작은 값을 한 그래프에서 효과적으로 표현 가능
상대적 변화(비율)를 쉽게 파악 가능
단점: 실제 값을 직접적으로 읽기 어려움'''

### 거래 유형별 사기 발생률 비교

In [ ]:
# MerchantID별 사기 거래 건수 계산
fraudByMerchant = fraud['MerchantID'].value_counts()

# 상위 10개 사기 다발 거래 유형 출력
print("\n=== Top 10 Merchant Types with Most Fraud Transactions ===")
print("\nMerchant Type | Number of Frauds")
print("-" * 40)
for merchant, count in fraudByMerchant.head(10).items():
    print(f"MerchantID {merchant}: {count}")

# 시각화
plt.figure(figsize=(12, 6))
fraudByMerchant.head(10).plot(kind='bar')
plt.title('Top 10 Merchant Types by Fraud Transactions')
plt.xlabel('MerchantID')
plt.ylabel('Number of Fraud Transactions')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# 전체 거래 대비 사기 거래 비율 계산
total_by_merchant = orgDf['MerchantID'].value_counts()
fraud_rate_by_merchant = (fraudByMerchant / total_by_merchant * 100).round(2)

print("\n=== Top 10 Merchant Types by Fraud Rate ===")
print("\nMerchant Type | Fraud Rate (%)")
print("-" * 40)
for merchant, rate in fraud_rate_by_merchant.sort_values(ascending=False).head(10).items():
    print(f"MerchantID {merchant}: {rate}%")


### 사기 거래가 자주 발생하는 지역 및 시간대 식별

In [ ]:
# TransactionDate에서 시간 추출 (시간별 분석)
fraud['Hour'] = pd.to_datetime(fraud['TransactionDate']).dt.hour

# 시간별 사기 거래 건수 계산
fraudByHour = fraud['Hour'].value_counts().sort_index()

# 가장 많은 사기가 발생한 시간 찾기
peakHour = fraudByHour.idxmax()
peakCount = fraudByHour.max()
print(f"사기 거래가 가장 많이 발생한 시간: {peakHour}시")
print(f"해당 시간대의 사기 거래 건수: {peakCount}건")

# 요일별 분석
fraud['DayOfWeek'] = pd.to_datetime(fraud['TransactionDate']).dt.day_name()
fraudByDay = fraud['DayOfWeek'].value_counts()

print("\n요일별 사기 거래 건수:")
print(fraudByDay)

# 월별 분석
fraud['Month'] = pd.to_datetime(fraud['TransactionDate']).dt.month
fraudByMonth = fraud['Month'].value_counts().sort_index()

print("\n월별 사기 거래 건수:")
print(fraudByMonth)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 서브플롯 설정
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

# 1. 시간별 사기 거래 시각화
fraudByHour.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Fraud Transactions by Hour')
ax1.set_xlabel('Hour')
ax1.set_ylabel('Number of Frauds')
ax1.grid(True)

# 최대값 표시
ax1.text(peakHour, peakCount, f'{peakCount}', 
         ha='center', va='bottom')

# 2. 요일별 사기 거래 시각화
fraudByDay.plot(kind='bar', ax=ax2, color='lightgreen')
ax2.set_title('Fraud Transactions by Day')
ax2.set_xlabel('Day of Week')
ax2.set_ylabel('Number of Frauds')
ax2.grid(True)

# 최대값 표시
max_day = fraudByDay.max()
max_day_idx = fraudByDay.idxmax()
ax2.text(fraudByDay.index.get_loc(max_day_idx), max_day, 
         f'{max_day}', ha='center', va='bottom')

# 3. 월별 사기 거래 시각화
fraudByMonth.plot(kind='bar', ax=ax3, color='salmon')
ax3.set_title('Fraud Transactions by Month')
ax3.set_xlabel('Month')
ax3.set_ylabel('Number of Frauds')
ax3.grid(True)

# 최대값 표시
max_month = fraudByMonth.max()
max_month_idx = fraudByMonth.idxmax()
ax3.text(fraudByMonth.index.get_loc(max_month_idx), max_month, 
         f'{max_month}', ha='center', va='bottom')

# 전체 레이아웃 조정
plt.tight_layout()
plt.show()

# 히트맵으로 시간대와 요일의 관계 시각화
pivot_table = pd.crosstab(
    pd.to_datetime(fraud['TransactionDate']).dt.hour,
    pd.to_datetime(fraud['TransactionDate']).dt.day_name()
)

plt.figure(figsize=(12, 8))
sns.heatmap(pivot_table, cmap='YlOrRd', annot=True, fmt='d')
plt.title('Fraud Transactions Heatmap (Hour vs Day)')
plt.xlabel('Day of Week')
plt.ylabel('Hour')
plt.show()
